In [ ]:
import pandas as pd

In [ ]:
import constants
import constraints

from datagolf import datagolf
from fileman import fileman
from picklejar import PickleJar

In [ ]:
constants.pandas_settings()
urls = {
    'skills-decomp': 'https://feeds.datagolf.com/preds/player-decompositions?tour={tour}&file_format={file_format}&key=6a626b6c312c0d33cfe157d614b5',
    'dfs-projections': 'https://feeds.datagolf.com/preds/fantasy-projection-defaults?tour={tour}&site={site}&slate={slate}&file_format={file_format}&key=6a626b6c312c0d33cfe157d614b5'
}

In [ ]:
skillsd=datagolf.load_skills() #.head(10)
skillsr=datagolf.load_skills(decomp=False)

master=skillsd.merge(skillsr)
order=list(master.columns)
master['delta']=master['final']/master['baseline']

master = master.loc[:, order[:3]+['delta']+order[3:]]

In [92]:
master.head(10)

,name,baseline,final,delta,category-sg,true-sg,driving-accuracy,total-fit,drv-acc,drv-dist,sg-app,sg-arg,sg-ott,sg-putt,sg-total
0,Sungjae Im,1.10,1.42,1.29,0.22,0.07,0.10,0.05,0.05,3.49,0.45,0.23,0.62,0.08,1.39
1,Tony Finau,1.44,1.23,0.86,-0.04,-0.02,-0.05,-0.14,-0.03,8.49,0.57,0.17,0.42,0.16,1.32
2,Hideki Matsuyama,1.35,1.21,0.90,-0.02,-0.04,0.02,-0.05,0.01,3.59,0.76,0.21,0.28,0.03,1.27
3,Cameron Tringale,0.83,0.87,1.06,-0.04,0.04,0.02,-0.02,0.01,1.37,0.30,0.10,0.11,0.30,0.80
4,Brendan Steele,0.64,0.81,1.27,0.17,-0.02,0.07,0.03,0.04,6.72,0.48,-0.09,0.70,-0.30,0.80
5,Adam Hadwin,0.76,0.81,1.06,-0.07,0.03,0.08,0.05,0.04,-1.77,0.19,0.30,0.00,0.22,0.70
6,Maverick McNealy,0.87,0.80,0.93,-0.04,0.04,0.01,-0.07,0.01,7.64,0.16,0.09,0.28,0.36,0.89
7,Davis Riley,0.71,0.75,1.05,0.08,0.02,0.04,-0.02,0.02,7.48,0.37,-0.01,0.38,0.07,0.81
8,Adam Long,0.62,0.69,1.11,-0.10,-0.02,0.13,0.16,0.07,-7.44,0.02,0.02,0.13,0.30,0.47
9,Cameron Davis,0.63,0.65,1.03,0.05,0.06,-0.08,-0.18,-0.04,12.38,0.35,-0.02,0.30,0.14,0.78


In [ ]:
def top_thresh(skill, num):
    return master.loc[master[skill]>=master[skill].describe(percentiles=(0.25, 0.33, 0.5, 0.66, 0.75, 0.82))[f'{num}%'], 'name'].values.tolist()

In [ ]:
threshold = 82

In [ ]:
def cycle():
    counter=top_thresh('delta',threshold)
    for col in list(filter(lambda x: x not in ('sg-putt', 'drv-dist'), list(master.columns)[4:])):
        counter+=top_thresh(col,threshold)
        
    count=dict()
    for name in counter:
        count[name] = count.get(name,0)+1
        
    return dict(sorted(count.items(), key=lambda x: x[1], reverse=True))

In [ ]:
results=cycle()
list(results.keys())
# results

In [ ]:
len(results)

In [ ]:
datagolf.load_dfs().head(10)

In [94]:
datagolf.load_baselines() #.head(25)

,name,make-cut,1st(%),2nd(%),3rd(%),top-10(%),top-20(%)
0,Sungjae Im,0.82,0.07,0.12,0.16,0.36,0.52
1,Hideki Matsuyama,0.81,0.06,0.11,0.15,0.33,0.50
2,Tony Finau,0.80,0.05,0.10,0.14,0.32,0.49
3,Maverick McNealy,0.75,0.03,0.06,0.09,0.24,0.39
4,Brendan Steele,0.73,0.03,0.06,0.09,0.22,0.37
5,Sahith Theegala,0.74,0.03,0.05,0.07,0.21,0.36
6,Cameron Tringale,0.73,0.03,0.05,0.07,0.21,0.36
7,Davis Riley,0.73,0.03,0.05,0.08,0.21,0.36
8,Cameron Davis,0.72,0.03,0.05,0.08,0.21,0.35
9,Adam Hadwin,0.72,0.03,0.05,0.07,0.20,0.34
